In [2]:
from EDTpy.ChipElements import *
import numpy as np
import copy
from EDTpy import settings

import EDTpy
from EDTpy import EmptyGeometry
from EDTpy import EmptyPath
from EDTpy.settings import *
import gdspy

In [159]:
line_a=6
line_b=6

In [160]:
class Marker(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        
        self + gdspy.Rectangle([-b/2-b/10, -b/2-b/10],
                               [b/2+b/10, b/2+b/10], layer = 0)
        
        mark = gdspy.boolean(gdspy.Rectangle([-b/2, -a/2],[b/2, a/2]), gdspy.Rectangle([-a/2, -b/2],
                               [a/2, b/2]), 'or', layer = 0)
        self - mark
        self.add_port([0, 0], 0)

In [161]:
class Planar_inductor(EmptyGeometry):
    default_values = {
        "a": line_a,
        "b": line_b,
        "gap": 10,
        "N_sp": 12,
        "width": 200,
        "leg": 0,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Planar_inductor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        leg = values['leg']
        layer = values['layer']

        self + gdspy.Rectangle([0, w+b],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w-b], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b+b+a, a/2], layer = 0)
        
        self - gdspy.Rectangle([2*b+b, a/2],
                               [2*b+b+a, w+a/2-gap], layer = 0)

        for i in range(N_sp):
            self - gdspy.Rectangle([2*b+b+a+(2*a+2*b)*(i), w+a/2-gap],
                                   [2*b+b+a+b+(2*a+2*b)*(i), w-gap-a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i)+b, -w-a/2+gap+a], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+a+b+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)

        self + gdspy.Rectangle([2*b+b+(2*a+2*b)*(N_sp), a/2],
                               [2*b+b+a+(2*a+2*b)*(N_sp), w+a/2-gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        #leg for connection
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)  

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [162]:
class Planar_capasitor(EmptyGeometry):
    default_values = {
        "a": line_a,
        "b": line_b,
        "gap": 10,
        "N_sp": 15,
        "width": 120,
        "layer": 0,
        "leg": 0
    }

    def _drawing(self, values):
        self.name = "Planar_capasitor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        layer = values['layer']
        leg = values['leg']

        self + gdspy.Rectangle([0, w],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b, a/2], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([2*b+(2*a+2*b)*(i), -a/2],
                                   [2*b+(2*a+2*b)*(i+1), a/2], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), w],
                                   [2*b+b+a+(2*a+2*b)*(i), a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), -w],
                                   [2*b+b+a+(2*a+2*b)*(i), -a/2-gap], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), -w+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -a/2], layer = 0)
            
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+2*b, a/2], layer = 0)
                    
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, w],
                               [(a+b)*2*(N_sp)+b*2+a+b, a/2+gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, -w],
                               [(a+b)*2*(N_sp)+b*2+a+b, -a/2-gap], layer = 0)
        
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [174]:
class Contact_pad_spiral(EmptyGeometry):
    default_values = {
        "a": line_a,
        "b": line_b,
        "N_sp": 6,
        "contact_pad_size": 250,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Contact_pad_spiral"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']

        self + gdspy.Rectangle([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, (contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)],
                                 [-(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3), -(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)], layer = 0)

        
        self - gdspy.Rectangle([-contact_pad_size/2, -contact_pad_size/2],
                                 [contact_pad_size/2, contact_pad_size/2], layer = 0)

        self - gdspy.Rectangle([-a/2, contact_pad_size/2],
                                 [a/2, contact_pad_size/2+b], layer = 0)
        self - gdspy.Rectangle([-a/2, contact_pad_size/2+b],
                                 [contact_pad_size/2+b, contact_pad_size/2+b+a], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(i), contact_pad_size/2+b+a+(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [-contact_pad_size/2-b-a-(a+b)*(i), +contact_pad_size/2+b+(a+b)*(i+1)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-a-(a+b)*(i), contact_pad_size/2+b+(a+b)*(i+1)],
                                     [contact_pad_size/2+b+(a+b)*(i+1), contact_pad_size/2+b+a+(a+b)*(i+1)], layer = 0)

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), contact_pad_size/2+b+a+(a+b)*(N_sp)],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp), a/2], layer = 0)  

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), a/2],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, -a/2], layer = 0)  

        self.add_port([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, 0], 0)
        self.add_port([0, 0], 180)

In [175]:
class LCLC(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
        "w": 270,
        "N_sp": 10,
        "N_c1": 10,
        "N_ind": 10,
        "N_c2": 10,
        "line": 6,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        w=values['w']

        N_sp=values['N_sp']
        N_c1=values['N_c1']
        N_ind=values['N_ind']
        N_c2=values['N_c2']
        
        line=values['line']
        
        spiral = Contact_pad_spiral(N_sp=N_sp)
#         spiral.merge_with(self.ports[9], 1)
        self + spiral
        cap = Planar_capasitor(width=w, N_sp=N_c1, leg = 0, a=line, b=line)
        cap.merge_with(spiral.ports[0], 0)
        self + cap
        ind_oo = Planar_inductor(width=w, N_sp=N_ind, leg = 0, a=line, b=line)
        ind_oo.merge_with(cap.ports[1], 0)
        self + ind_oo
        cap_o1o = Planar_capasitor(width=w, N_sp = N_c2, leg = 0, a=line, b=line)
        cap_o1o.merge_with(ind_oo.ports[1], 0)
        self + cap_o1o
        x = cap_o1o.ports[1].position[0]
        y = cap_o1o.ports[1].position[1]
        self.add_port([0, 0], 0)
        self.add_port([x, y], 0)
        

In [176]:
class LCL(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
        "w": 270,
        "N_sp": 10,
        "N_c1": 10,
        "N_ind": 10,
        "line": 6,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        w=values['w']

        N_sp=values['N_sp']
        N_c1=values['N_c1']
        N_ind=values['N_ind']
        
        line=values['line']
        
        spiral = Contact_pad_spiral(N_sp=N_sp, a=line, b=line)
#         spiral.merge_with(self.ports[9], 1)
        self + spiral
        cap = Planar_capasitor(width=w, N_sp=N_c1, leg = 0, a=line, b=line)
        cap.merge_with(spiral.ports[0], 0)
        self + cap
        ind_oo = Planar_inductor(width=w, N_sp=N_ind, leg = 0, a=line, b=line)
        ind_oo.merge_with(cap.ports[1], 0)
        self + ind_oo
#         cap_o1o = Planar_capasitor(width=w, N_sp = N_c2, leg = 0)
#         cap_o1o.merge_with(ind_oo.ports[1], 0)
#         self + cap_o1o
        x = ind_oo.ports[1].position[0]
        y = ind_oo.ports[1].position[1]
        self.add_port([0, 0], 0)
        self.add_port([x, y], 0)

In [177]:
class LC(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
        "w": 270,
        "N_sp": 10,
        "N_c1": 10,
        "line": 6,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        w=values['w']

        N_sp=values['N_sp']
        N_c1=values['N_c1']
        line=values['line']
        
        spiral = Contact_pad_spiral(N_sp=N_sp)
#         spiral.merge_with(self.ports[9], 1)
        self + spiral
        cap = Planar_capasitor(width=w, N_sp=N_c1, leg = 0, a=line, b=line)
        cap.merge_with(spiral.ports[0], 0)
        self + cap
#         ind_oo = Planar_inductor(width=w, N_sp=N_ind, leg = 0)
#         ind_oo.merge_with(cap.ports[1], 0)
#         self + ind_oo
#         cap_o1o = Planar_capasitor(width=w, N_sp = N_c2, leg = 0)
#         cap_o1o.merge_with(ind_oo.ports[1], 0)
#         self + cap_o1o
        x = cap.ports[1].position[0]
        y = cap.ports[1].position[1]
        self.add_port([0, 0], 0)
        self.add_port([x, y], 0)

In [208]:
res_R = 45
CPW.default_values['S'] = 10
CPW.default_values['W'] = 6
CPW.default_values['layer'] = 0

sketch = EDTpy.EmptySketch()
sketch + gdspy.Rectangle([-5000/2, -5000/2],
                         [5000/2, 5000/2], layer = 0)

sketch - gdspy.Rectangle([-5000/2 + 50, -5000/2 + 50],
                         [5000/2 - 50, 5000/2 - 50], layer = 0)

sketch.add_port([-2100, 2100], 0) #Marker port  --  0
marker = Marker()
marker.merge_with(sketch.ports[0], 0)
sketch.add_geometry(marker)
sketch.circular_array(marker)


sketch.add_port([-1600, 2000], 90) #DC port  -- 1
lclc = LCLC()
lclc.merge_with(sketch.ports[1], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F1", 70, (-1250, 1950))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, 150], [500, 150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, 680],
                         [-2150-150, 830], layer = 0)
sketch - gdspy.Rectangle([-2150-10, 680+10],
                         [-2150-150+10, 830-10], layer = 0)
sketch.assemble()

base_x = -700
base_y = 2000
sketch.add_port([base_x, base_y], 90) #DC port  -- 2
lclc = LCLC(N_sp=15, N_c1=10, N_ind=10, N_c2=10)
lclc.merge_with(sketch.ports[2], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F2", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, 150+200*1], [1500, 150+200*1]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, 720-290],
                         [-2150-150, 870-290], layer = 0)
sketch - gdspy.Rectangle([-2150-10, 720-290+10],
                         [-2150-150+10, 870-290-10], layer = 0)
sketch.assemble()


base_x = 200
base_y = 2000
sketch.add_port([base_x, base_y], 90) #DC port  -- 3
lclc = LCLC(N_sp=10, N_c1=20, N_ind=10, N_c2=10)
lclc.merge_with(sketch.ports[3], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F3", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, 150+250], [2300, 150+250]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, 750-290*2],
                         [-2150-150, 900-290*2], layer = 0)
sketch - gdspy.Rectangle([-2150-10, 750-290*2+10],
                         [-2150-150+10, 900-290*2-10], layer = 0)
sketch.assemble()



base_x = -1600
base_y = -2000
sketch.add_port([base_x, base_y], -90) #DC port  -- 4
lclc = LCLC(N_sp=20, N_c1=25, N_ind=25, N_c2=35, line=2)
lclc.merge_with(sketch.ports[4], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F4", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, -150], [500, -150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, -650],
                         [-2150-150, -800], layer = 0)
sketch - gdspy.Rectangle([-2150-10, -650-10],
                         [-2150-150+10, -800+10], layer = 0)
sketch.assemble()

base_x = -700
base_y = -2000
sketch.add_port([base_x, base_y], -90) #DC port  -- 5
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=2)
lclc.merge_with(sketch.ports[5], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F5", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, -150], [1400, -150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, -200],
                         [-2150-150, -350], layer = 0)
sketch - gdspy.Rectangle([-2150-10, -200-10],
                         [-2150-150+10, -350+10], layer = 0)
sketch.assemble()


base_x = 200
base_y = -2000
sketch.add_port([base_x, base_y], -90) #DC port  -- 6
lclc = LCLC(N_sp=25, N_c1=50, N_ind=50, N_c2=75, line=2)
lclc.merge_with(sketch.ports[6], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("F6", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, -150], [2300, -150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([-2150, 0],
                         [-2150-150, -150], layer = 0)
sketch - gdspy.Rectangle([-2150-10, 0-10],
                         [-2150-150+10, -150+10], layer = 0)
sketch.assemble()



base_x = 1300
base_y = 2000
sketch.add_port([base_x, base_y], 90) #DC port  -- 7
lclc = LCL(N_sp=15, N_c1=25, N_ind=30)
lclc.merge_with(sketch.ports[7], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("T1", 70, (base_x+370, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, -150], [800, -150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([2150, 50],
                         [2150+150, 50+150], layer = 0)
sketch - gdspy.Rectangle([2150+10, 50+10],
                         [2150+150-10, 50+150-10], layer = 0)
sketch.assemble()


base_x = 1300
base_y = -1700
sketch.add_port([base_x, base_y], -90) #DC port  -- 8
lclc = LC(N_sp=25, N_c1=35)
lclc.merge_with(sketch.ports[8], 0)
sketch.add_geometry(lclc)
text = gdspy.Text("T2", 70, (base_x+500, base_y-50))
# inv = gdspy.boolean(el, text, "not")
sketch + text
path_gate = [[0, 0], [0, 150], [800, 150]]
res_gate = CPW(path_gate, res_R, S=6)
res_gate.merge_with(lclc.ports[1], 0)
sketch.add_geometry(res_gate)
sketch + gdspy.Rectangle([2150, -150],
                         [2150+150, -150-150], layer = 0)
sketch - gdspy.Rectangle([2150+10, -150-10],
                         [2150+150-10, -150-150+10], layer = 0)
sketch.assemble()

sketch.save_gds("C:/Users/mukha/Documents/Aalto/Filters/LC_filters_test")

# sketch.show()

Assembling Empty Sketch:   0%|                                                                                                                                                                                                     | 0/10 [00:00<?, ?it/s]

Assembling Empty Sketch:  94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎           | 15/16 [00:00<00:00, 70.21it/s]


Assembling Empty Sketch:   0%|                                                                                                                                                                                                     | 0/18 [00:00<?, ?it/s]


Assembling Empty Sketch:  28%|████████████████████████████████████████████████████▌                                                                                                                                        | 5/18 [00:00<00:00, 49

Assembling Empty Sketch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00, 46.27it/s]